In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

### Carga de datos corregidos

In [10]:
# %timeit sirve para evaluar el tiempo de ejecucion
print("------------------");
df_postulantes_educacion = pd.read_csv('../csv/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
print("###", end='');
df_postulantes_genero_edad = pd.read_csv('../csv/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad_fix.csv')
print("###", end='');
df_vistas_general = pd.read_csv('../csv/datos_navent_fiuba/fiuba_3_vistas.csv')
print("###", end='');
df_postulaciones = pd.read_csv('../csv/datos_navent_fiuba/fiuba_4_postulaciones.csv')
print("###", end='');
df_avisos_online = pd.read_csv('../csv/datos_navent_fiuba/fiuba_5_avisos_online.csv')
print("###", end='');
df_avisos_detalle = pd.read_csv('../csv/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
print("###");
print("------------------");

------------------
##################
------------------
